In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image
import pylab

### Load Data

In [12]:
X = []
Y = []
df = np.array(pd.read_csv("train_img.csv"))
for i in df:
    X.append(np.array(Image.open(f"train_images/train_images/{i[0]}").resize((128,128))))
    Y.append(i[1])

In [13]:
Y_mapping = {}
l = 0
for i in range(len(Y)):
    try:
        tmp = Y_mapping[Y[i]]
        Y[i] = tmp
    except:
        Y_mapping[Y[i]] = l
        Y[i] = l
        l+=1

In [16]:
for i in range(len(Y)):
    tmp = np.zeros_like([0]*(l+1))
    tmp[Y[i]] = 1
    Y[i] = tmp

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(4)    
])

2022-12-30 18:10:36.516105: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-30 18:10:36.516970: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-30 18:10:36.517115: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-F63Q4HB): /proc/driver/nvidia/version does not exist
2022-12-30 18:10:36.520552: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
